In [ ]:
from IPython.display import display
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re

# Week 2: Working with Big Datasets

## Motivation

**Publication of crawling papers by year**

![Publication of crawling papers by year](images/publication_crawling_papers_by_year.png)*Source*: Claussen, Jörg and Peukert, Christian, **Obtaining Data from the Internet: A Guide to Data Crawling in Management Research** (June 2019). Available at SSRN: https://ssrn.com/abstract=3403799 or http://dx.doi.org/10.2139/ssrn.3403799
    

**General objective of the notebook**: construct a dataset with the **tweets** of the current U.S. members of Congress (Senate + House) with information on their party **affiliation**

**Three sources of data**:
 1. **List of U.S. representatives**: **webscraped** from [ballotpedia](https://ballotpedia.org/List_of_current_members_of_the_U.S._Congress)
 2. **Twitter accounts** of the U.S. representative. From a [hand-labeld dataset](https://github.com/vegetable68/Midterm-2018-candidates) compiled by Yiqing Hua for all candidates.
 3. **Tweets** published on the twitter accounts
 
**2 merge operations**:
- 1+2: select only the elected representative among the candidates present in 2
- 3+2+1: tweets associated with their author + party affiliation

## Screen scraping

What is webscraping ?

<img src="images/screenscraping.png">

Source: [SICSS](https://compsocialscience.github.io) 

Points to keep in mind:
- It may or may not be legal
- Webscraping is tedious and frustrating

Main challenges:
- Variety of websites and webpages
- Durability of code as website constantly changes

## Typical Steps of Webscraping

### Exploring the Website

We will scrape the list of current members of the U.S. Congress because it will be useful later in the class!
<img src="images/ballotpedia.png">

Source: [ballotpedia website](https://ballotpedia.org/List_of_current_members_of_the_U.S._Congress) 

### Understanding URLs
- Base URL: https://ballotpedia.org/List_of_current_members_of_the_U.S._Congress
- More complex URL with query parameter https://ballotpedia.org/wiki/index.php?search=jerry&searchToken=elnan6bftyqukadgu8xb2rtbg
    - query parameter=`p?search=jerry`
    - can be used to crawl websites if you have a list of queries that you want to loop over (e.g. dates, localities...)
    - query structure:
        - *Start*: `?`
        - *Information*: pieces of information constituting one query parameter are encoded in key-value pairs, where related keys and values are joined together by an equals sign (key=value). 
        - *Separator*: `&` -> if multiple query parameters 
        
Other example of URL: https://opendata.swiss/en/dataset?political_level=commune&q=health. Try to change the search and selection parameters and observe how that affects your URL. 

### Inspect the site Using Developer Tools
We use the `inspect` function (right click) to access the underlying HTML interactively. 
<img src="images/ballotpedia_inspect.png">





**R users** 

The logic shown hereafter has its direct equivalent in `R`. See [this post](https://towardsdatascience.com/web-scraping-tutorial-in-r-5e71fd107f32) for examples of the most useful functions. 

### HTML parsing
In this example, we scrape **static HTML content**: the server that hosts the site sends back HTML documents that already contain all the data you’ll get to see as a user.

In [ ]:
import urllib # Python's module for accessing web pages
url='https://ballotpedia.org/List_of_current_members_of_the_U.S._Congress'

In [ ]:
page = urllib.request.urlopen(url) # open the web page
html = page.read() # read web page contents as a string
print("-- first 400 characters --", html[:400]) 
print("-- last 400 characters --", html[-400:])
print("-- length of string --", len(html))

In [ ]:
# Parse raw HTML
from bs4 import BeautifulSoup # package for parsing HTML
soup = BeautifulSoup(html, 'html.parser') # parse html of web page
print("-- title item:", soup.title) 

In [ ]:
print(soup.prettify())

In [ ]:
# extract text
text = soup.get_text() # get text (remove HTML markup)
lines = text.splitlines() # split string into separate lines
print("-- Number of lines:", len(lines))

In [ ]:
lines = [line for line in lines if line != ''] # drop empty lines
print("-- Number of lines (after dropping empty lines):", len(lines))
print("-- The first 20 lines:", lines[:20])

### Scraping a table

#### Find Elements by ID
using the `find` function 

In [ ]:
print(soup.find(id="mw-content-text"))

In [ ]:
results=soup.find(id="mw-content-text")
print(results.prettify())

#### Find Elements by HTML Class Name

In [ ]:
soup.find('table', class_='wikitable sortable jquery-tablesorter')

`find_all` is often more useful than `find`.

In [ ]:
tb = soup.find_all('table', class_='wikitable sortable jquery-tablesorter')
len(tb)

In [ ]:
senate=tb[0] # first element 
print(senate.find_all('tr')[1]) # a row of the table

#### Extract Text From HTML Elements

In [ ]:
import re # for regular expressions

Cleaning the row: usnig the `get_text` function

In [ ]:
row=senate.find_all('tr')[1]

print("-- Number of rows: {}".format(len(senate.find_all('tr'))))

# Using the a of the first 2 cells
for cell in row.find_all('a'):
    print(cell.get_text())

# For the 2 last cells:
for cell in row.find_all('p')[2:4]:
    print(re.sub('\n', '', cell.get_text().lstrip())) # little text trick: wait for the class on text-as-data!

Loop over all rows:

In [ ]:
df_senate=pd.DataFrame() # empty dataframe in which the cleaned rows will be stored

for row in senate.find_all('tr'):
    row_dict=dict() # empty dictionary in which the cleaned cells are stored
    i=0
    # Using the a of the first 2 cells
    for cell in row.find_all('a'):
        row_dict[i]=[cell.get_text()]
        i=i+1
    # For the 2 last cells:
    for cell in row.find_all('p')[2:4]:
        row_dict[i]=[re.sub('\n', '', cell.get_text().lstrip())]
        i=i+1
    df_row=pd.DataFrame.from_dict(row_dict, orient='columns') # row_dict -> dataframe
    df_senate=pd.concat([df_senate, df_row]) # append the df_row 

In [ ]:
df_senate=df_senate.rename(columns={0:'Officeholder name', 1: 'Office title', 2: 'Date assumed office', 3: 'Party affiliation'})
df_senate

In [ ]:
df_senate['Party affiliation'].value_counts()

#### Exercise: construct a dataframe containing the table on the House 

The house composition is the second table of the page

### Going further

There are also **dynamic websites**: the server does not always send back HTML, but your browser also receive and interpret JavaScript code that you cannot retreive from the HTML. You receive JavaScript code that you cannot parse using `beautiful soup` but that you would need to execute like a browser does. 

Solutions: 
- Use `requests-html` 
- Simulate a browser using [selenium](https://selenium-python.readthedocs.io/) 

## Data on politician with info on party and twitter accounts

We need to find (or build from scratch) a data with information on the politician. Most importantly, we need a link to their twitter account and their party affiliation. 

Such a dataset has been constructed by Yiqing Hua (Cornell Tech) for  US. midterm election 2018 candidates with their twitter handles data from https://github.com/vegetable68/Midterm-2018-candidates

Data = full list of candidates running for House and Senate, as well as gubernatorial candidates from Ballotpedia

In [ ]:
# read file with pandas (stored on github)
df = pd.read_csv('https://raw.githubusercontent.com/vegetable68/Midterm-2018-candidates/master/candidates.csv')
df.head()

In [ ]:
df['party'].value_counts()

In [ ]:
df['gender'].value_counts()

In [ ]:
df_candidates=df[['candidate_name', 'party', 'twitter handle']]

Merge with House and Senate data : only keeps the elected candidates

In [ ]:
df_congress= pd.concat([df_house, df_senate])
df_congress.shape

In [ ]:
print(df_candidates.shape)
print("Number of unique candidate names", len(df_candidates['candidate_name'].unique()))

In [ ]:
df_merged_all=pd.merge(df_congress, df_candidates, right_on='candidate_name', left_on='Officeholder name')

In [ ]:
print("--Result of the merge:")
print("Number of twitter accounts from candidates:", df_candidates.shape[0])
print("Number of twitter accounts from US representative:", df_merged_all.shape[0])
print("Correspond to {} politicians (often having 2 accounts)".format(len(df_merged_all['Officeholder name'].unique())))
print("Number US representative:", df_congress.shape[0])
print("Share of US representative with a twitter account:", len(df_merged_all['Officeholder name'].unique())/df_congress.shape[0])

List of tweeter accounts, useful for the following task

In [ ]:
account_list = df_merged_all['twitter handle'].tolist()
print('First 3 elements:', account_list[:3])
print('Number of twitter account studied:', len(account_list))

## Application Programming Interfaces (API)
### What Is an API?

**APIs are tools for building apps or other forms of software that help people access certain parts of large databases**

The website [Programmable Web](https://www.programmableweb.com/apis/directory) lists more than 225,353 API from sites as diverse as Google, Amazon, YouTube, the New York Times, del.icio.us, LinkedIn, and many others.

<img src="images/growth_in_web_api.png">

Source: [Programmable Web](https://www.programmableweb.com/news/apis-show-faster-growth-rate-2019-previous-years/research/2019/07/17) 


### How Does an API Work?

Better than webscraping if possible because: 
- More stable than webpages
- No HTML but already structured data (e.g. in `json`)

### API Credentials
In order to prevent software developer to collect huge amount of individual data, many APIs require you to obtain “credentials” or codes/passwords that identify you and determine which types of data you are allowed to access. 

### Rate Limiting
The credentials not only define what type of information we are allowed to access, but also how often we are allowed to make requests for such data. 

## Why Using Twitter's API?

- Increasingly used in Political Sciences and Economics 
    - Allyson L. Benton & Andrew Q. Philips, 2020. **"Does the @realDonaldTrump Really Matter to Financial Markets?,"** *American Journal of Political Science*, John Wiley & Sons, vol. 64(1), pages 169-190, January. [Website](https://onlinelibrary.wiley.com/doi/10.1111/ajps.12491)
    - Petrova Maria Sen Ananya and Yildirim Pinar, **Social Media and Political Donations: New Technology and Incumbency Advantage in the United States** (September 8, 2016). [SSRN](https://ssrn.com/abstract=2836323)
    - **Analyzing Polarization in Social Media: Method and Application to Tweets on 21 Mass Shootings** by Dorottya Demszky, Nikhil Garg, Matthew Gentzkow,  Rob Voigt, James Zou, Jesse M. Shapiro, and Dan Jurafsky, 17th Annual Conference of the North American Chapter of the Association for Computational Linguistics (NAACL). June 2019. [arxiv](https://arxiv.org/abs/1904.01596)
- As an example for using an API


## An Example with Twitter’s API

### How to apply for a developer account
Developers need first to have a twitter account: this tutorial assumes that it is already the case. 

How to obtain credentials from Twitter that will allow you to make API calls? 

1. create an account (https://apps.twitter.com) in order to receive credentials  
2. create a developer account by clicking ''Apply for a developer account''. 
3. confirm your email address or add a mobile phone number (two-factor authentication helps Twitter prevent people from obtaining a large number of different credentials using multiple accounts that could be use to collect large amounts of data without being rate limited—or, for other nefarious purposes such as creating armies of bots that produce spam or attempt to influence elections.)
4. answer series of questions about how you want to use Twitter’s API & accept terms of services
5. Once you accept the terms, your app developer request will go under review by Twitter. Then it takes time (1-2 days to a week)

### Create an Application & get your authentification details
1. Once the developer account is approved, go to your profile tab and select Apps. Create an app and fill in the details.
2. Click on `details`
3. Click on `Keys and tokens`. This is where you get the relevant keys (you will have to regenerate and copy the tokens):
    - API key
    - API secret key
    - Access token
    - Access token secret


After registering to the Twitter API, you get:

In [ ]:
 #this you get when you make create an application on twitter as a dev
consumer_key="YOURKEYHERE"
consumer_secret="YOURSECRETHERE"
access_token="YOURACCESSTOKENHERE"
access_token_secret="YOURACCESSTOKENSECRETHERE"

## Accessing the Twitter API using `tweepy`

We use the `tweepy` package (documentation: https://tweepy.readthedocs.io/en/latest/). Tweepy is an *An easy-to-use Python library for accessing the Twitter API.*

R users can use [rtweet](https://rtweet.info/), a similar package. 

Twitter requires all requests to use `OAuth` for authentication

In [ ]:
import tweepy
from tweepy import OAuthHandler

Authenticate to Twitter

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret) #creating an OAuthHandler instance
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

We specify `wait_on_rate_limit_notify==True` & `wait_on_rate_limit=True`. The API method will wait once you’ve reached your rate limit and prints out a message

In [ ]:
# test authentication
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

For an extensive list of the methods available, see the [API Reference page](https://tweepy.readthedocs.io/en/latest/api.html#api-reference). There are several types of methods. The following methods enable you to access twitter content:
- Timeline methods return a list of  `status` objects
- Status methods return a `status` object
- User methods return `user` object or a list of `user` objects. 
- Favorite methods: return a list of  `status` objects

For some methods, you can interact with twitter:
- Friendship Methods return a `user` object, by example:
    - `create_friendship`: creates a new friendship with the specified user ()
    
Let's review some useful methods:

### Methods returning a `status` object (or a list of objects)
#### Search method

If you seeking Twitter data to get conversations on a particular topic. This method returns a collection of relevant Tweets matching a specified query for all public tweets.


In [ ]:
# most recent tweets about ETH 
tweets = api.search(q="ETH Zürich", lang="en")
for tweet in tweets:
    print(tweet.text) # printing the first tweet

The `status` object:

In [ ]:
pprint(tweets[0]) # for the first tweet

In [ ]:
pprint(tweets[0]._api) # for the first tweet

In [ ]:
pprint(tweets[0]._json) # for the first tweet

#### Small introduction to `json` format and dictionaries

`JSON` (`JavaScript Object Notation`) is a popular data format used for representing structured data. See the chapter in the [Hitchhiker’s Guide to Python](https://docs.python-guide.org/scenarios/json/)

The object is a **dictionary**. Dictionaries are Python objects associating keys to values. Keys and Values can be any Python object: scalar, string, list, dictionaries... If a value is a dictionary, then the overall dictionary embed a hierachical structure.

See chapter 3.1 of [Python for Data Analysis](https://learning.oreilly.com/library/view/python-for-data/9781491957653/) for more on built_in data structures, including dictionaries. 

In [ ]:
empty_dict={} # dict are defined by curly braces
d1 = {'a' : 'some value', 'b' : [1, 2, 3, 4], 'c' : {'c1': 10, 'c2':20}}
d1['d']='more' # add a key-value pair in d1
print(d1)

In [ ]:
#navigating in the dictionary using the keys:
print(d1['a'])

In [ ]:
print(d1['c']['c2']) # works several time: a handy way to get to an element

In [ ]:
print(tweets[0]._json.keys()) # Keys of dictionary (for the first tweet)

**Exercice**  Access the screen name of the first tweet

#### home_timeline
Returns the 20 most recent statuses, including retweets, posted by the uthenticating user and that user’s friends. This is the equivalent of /timeline/home on the Web.

In [ ]:
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

#### user_timeline
The overall rate limit to this method is 100,000 calls during any single 24-hour period. That will translate to 100,000 users and their timeline posts (up to 200 most recent posts).

In [ ]:
timeline = api.user_timeline(user_id=46182536, count=2)
print(len(timeline))

### Methods returning a `user` object (or a list of objects)
- `me` returns the authenticated user's information

In [ ]:
api.me()

- The `get_user` method returns information about the specified user.

In [ ]:
target=account_list[0] #'JenniferWexton'
user = api.get_user(target) # argument = id, user_id, screen_name
pprint(user._json)

Some attributes of the `user` object:

In [ ]:
print("Name:", user.name)
print("Screen name:", user.screen_name)
print("Number of followers:" ,  user.followers_count)
pprint("description: " + user.description)
pprint("Number of tweets published: " + str(user.statuses_count))
pprint("friends_count: " + str(user.friends_count))

- `followers` returns the user's followers
- `search_users` searches for users

### A Friendship method: follower_ids
This method allows you to get most recent following of a particular user (use screen_name as parameter). 
It is useful if you want to get all the tweets on the timeline of a particular user. 

In [ ]:
followers=api.followers_ids(screen_name=target)
print(followers[0:10])

Fetch the first 10 tweets published by this account:

In [ ]:
tweets = api.user_timeline(screen_name = target, count = 10, include_rts = True)

### Looping over `account_list` 

Handling the rate limit imposed by the API

In [ ]:
import time
time.sleep(3) # wait for three seconds

In [ ]:
nb_tweets_by_target=2
print("We aim at fetching {} tweets".format(len(account_list)* nb_tweets_by_target))

In [ ]:
%%time 
# to get an idea of how long it takes

df_tweets=pd.DataFrame() # empty dataframe where the tweet will be saved

if len(account_list) > 0:
    
    # Restricting the search for the first 10 accounts
    for target in account_list[:10]:
        
        # try the following:
        try:
            # Fetch nb_tweets_by_target for target
            tweets = api.user_timeline(screen_name = target, count = nb_tweets_by_target, include_rts = False)
            
            # Put the tweets into a dataframe object
            tweet_count=0
            for tweet in tweets:
                # 1. Transform the json into a dataframe
                df_tweet=pd.DataFrame.from_dict(tweet._json, orient='index', columns=[tweet_count]) # , sleep_on_rate_limit=True
                # 2. adds screen name as a row
                df_tweet=df_tweet.append(pd.DataFrame({tweet_count:[target]}, 
                                                      index=['twitter handle']))
                # 3. Add the tweet dataframe to the df_tweets dataframe
                df_tweets=pd.concat([df_tweet, df_tweets], axis=1)
                
                # counting the number of target fetched
                tweet_count += 1 
                
            time.sleep(0.5)
            
        # except if TweepError arises
        except tweepy.TweepError: #the error arises when the user has protected tweets
            print("Failed to run the command on user {}, Skipping...".format(target))
            
        # except if RateLimitError arises
        except tweepy.RateLimitError:
            print("ressource usage limit: {} skipped".format(target))
            time.sleep(0.3)

In [ ]:
df_tweets=df_tweets.transpose() # Transpose the dataset
print(df_tweets.columns)
print(df_tweets.shape)

## Merge tweet and party affiliation on `twitter handle` 

In [ ]:
df_tweets_small=df_tweets[['text', 'created_at', 'retweet_count', 'favorite_count', 'twitter handle']] # 'user'
df_tweets_small.head()

In [ ]:
df_merged=pd.merge(df_tweets_small, df_merged_all,on='twitter handle')
df_merged.shape

<div class="alert alert-block alert-warning">
<i class="fa fa-warning"></i>&nbsp;<code>os</code> package
    <ul>
        <li> <code>os.getcwd()</code>: fetchs the current path
        </li>
        <li> <code>os.path.dirname()</code>: go back to the parent directory
        </li>
        <li> <code>os.path.join()</code>: concatenates several paths
        </li>
    </ul>
</div>

In [ ]:
import os
parent_path=os.path.dirname(os.getcwd()) # os.getcwd() fetchs the current path, 
data_path=os.path.join(parent_path, 'data')
print(data_path)

<div class="alert alert-block alert-warning">
<i class="fa fa-warning"></i>&nbsp;<code>pickle</code> format
    <ul>
        <li> Useful to store <code>python</code> objects 
        </li>
        <li> Well integrated in  <code>pandas</code> (using <code>to_pickle</code> and <code>read_pickle</code>)
        </li>
        <li> When the object is not a pandas Dataframe, use the <code>pickle</code> package
        </li>
    </ul>
</div>


In [ ]:
df_merged.to_pickle(data_path+'/tweet_labeled.pkl')
df_merged.to_csv(data_path+'/tweet_labeled.csv')

## Other example using API

Forecasts from the **Carbon Intensity API**: https://carbonintensity.org.uk/ (include CO2 emissions related to eletricity generation only).
See the API [documentation](https://carbon-intensity.github.io/api-definitions/#carbon-intensity-api-v2-0-0)

In [ ]:
import requests
headers = {
  'Accept': 'application/json'
}

In [ ]:
# Get Carbon Intensity data for current half hour
r = requests.get('https://api.carbonintensity.org.uk/intensity', params={}, headers = headers) 
print(r.json())

In [ ]:
# Get Carbon Intensity data for today
r = requests.get('https://api.carbonintensity.org.uk/intensity/date', params={}, headers = headers)
pprint(r.json())

In [ ]:
# Get Carbon Intensity factors for each fuel type
r = requests.get('https://api.carbonintensity.org.uk/intensity/factors', params={}, headers = headers)
pprint(r.json())

In [ ]:
# Get Carbon Intensity data for current half hour for GB regions
r = requests.get('https://api.carbonintensity.org.uk/regional', params={}, headers = headers)
pprint(r.json())

## Class survey
Please fill in this [short survey](https://framaforms.org/keep-start-stop-survey-1583156515) about the class. 

## What is not covered in the notebook

- If you struggle something and you need for your project, tell us and we can spend some time on it. For example:
    - Scraping dynamically-generated content